# LATAM Airlines

Scrapear el sitio de Despegar para averiguar datos de vuelos en función del **origen** y **destino**, **fecha** y **cabina**. La información que esperamos obtener de cada vuelo es:

- Precios(s) disponibles
- Horas de salida y llegada (duración)
- Información de las escalas

**NOTA**: en caso de no poder acceder (status_code = 403), buscar un sitio similar.

www.edestinos.com

In [69]:
import requests
from bs4 import BeautifulSoup

In [70]:
#url = 'https://www.edestinos.com/flights/select/oneway/ap/mvd/ap/for?departureDate=2020-12-10&pa=1&py=0&pc=0&pi=0&sc=economy'

In [71]:
url = 'https://www.latam.com/es_uy/apps/personas/booking?fecha1_dia=04&fecha1_anomes=2020-11&from_city1=MVD&to_city1=MAD&ida_vuelta=ida_vuelta&fecha2_dia=21&fecha2_anomes=2020-11&from_city2=MAD&to_city2=MVD&cabina=Y&nadults=1&nchildren=0&ninfants=0&app=deal-finder#/'

In [72]:
r = requests.get(url)

In [73]:
r.status_code

403

In [74]:
soup = BeautifulSoup(r.text, 'lxml')

In [75]:
# Importar Selenium (webdriver)

from selenium import webdriver

In [76]:
# Iniciar navegador automatico
options = webdriver.ChromeOptions()
options.add_argument('--incognito')

driver = webdriver.Chrome(executable_path='/Users/Martin/Documents/Proyectos/Airlines_scrapper/chromedriver', options=options)

In [77]:
driver.get(url)

In [80]:
# Extraer informacion de la pagina

vuelos = driver.find_elements_by_xpath('//li[@class="flight"]') # // buscar en todo el arbol
len(vuelos)

6

In [13]:
vuelo = vuelos[0]

In [ ]:
vuelo

**Extrayendo horarios y duración**

In [14]:
#Hora de partida
vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime') # . buscar desde allí hacia abajo

'11:18'

In [15]:
#Hora de llegada
vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')


'12:50'

In [16]:
# Duración de vuelo
vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')

'PT21H32M'

**Contando escalas e interactuando con elementos del sitio**

In [17]:
boton_escalas = vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')

In [18]:
boton_escalas.click() #click en boton

In [19]:
# Segmentos del contenido en donde se ven las escalas
segmentos = vuelo.find_elements_by_xpath('//div[@class="sc-hZSUBg gfeULV"]/div[@class="sc-cLQEGU hyoued"]')
# Sin '.' porque busco todo el album.
segmentos

[<selenium.webdriver.remote.webelement.WebElement (session="5feffdd59576557aa98daaa3562b2eb5", element="0abd0b35-b010-483c-a036-3c345dcf67d5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5feffdd59576557aa98daaa3562b2eb5", element="d5ac38f3-a698-45ee-897b-f805ce000406")>]

In [20]:
escalas = len(segmentos) - 1 # Porque cuenta desde salida/escala y escala/salida
print(f'El vuelo de MVD a MAD tiene {escalas} escala(s).')

El vuelo de MVD a MAD tiene 1 escala(s).


**Desafío: Escalas**

- **Escala #1**

In [21]:
segmento = segmentos[0]

In [22]:
# Aeropuerto de salida
segmento.find_element_by_xpath('.//div[@class="sc-iujRgT jEtESl"]/span[@class="sc-eTuwsz eumCTU"]/span[@class="sc-hXRMBi gVvErD"]').text 


'Carrasco Intl.'

In [23]:
# Hora de salida
segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/time').get_attribute('datetime')

'11:18'

In [24]:
segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/span[@class="sr-only"]').text

'Salida a las 11 Horas 18 Minutos, de Carrasco Intl.'

In [25]:
# Duración de vuelo a próxima escala
segmento.find_element_by_xpath('.//div[@class="sc-eXEjpC fqUnRK"]/span[@class="sc-esjQYD dMquDU"]/time').get_attribute('datetime')

'2:32'

In [26]:
try:    
    combinacion = segmento.find_element_by_xpath('//div[@class="sc-hMFtBS cfwWiO"]/span[@class="connection-label sc-hORach NXcGo"]').text
    cambio_avion = segmento.find_element_by_xpath('//div[@class="sc-hMFtBS cfwWiO"]/span[@class="connection-changes sc-hORach NXcGo"]').text
    print(f'{combinacion} -- {cambio_avion}')
except:
    print('Revise XPath.')
    pass



Conexión en Santiago de Chile -- (cambio de avión)


In [27]:
modelo_avion = segmento.find_element_by_xpath('//div[@class="airline-flight-details"]/span[@class="sc-gzOgki uTyOl"]').text
flota = segmento.find_element_by_xpath('//div[@class="airline-flight-details"]/b').text

print(f'Flota {flota} - {modelo_avion}')

Flota LA405 - Airbus 320-200


- **Escala #2**

In [28]:
segmento_2 = segmentos[1]

In [29]:
# Aeropuerto de salida
segmento_2.find_element_by_xpath('.//div[@class="sc-iujRgT jEtESl"]/span[@class="sc-eTuwsz eumCTU"]/span[@class="sc-hXRMBi gVvErD"]').text 

'A. Merino Benítez Intl.'

In [30]:
# Hora de salida
segmento_2.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/time').get_attribute('datetime')

'19:55'

In [31]:
segmento_2.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/span[@class="sr-only"]').text


'Salida a las 19 Horas 55 Minutos, de A. Merino Benítez Intl.'

In [32]:
# Duración de vuelo a próxima escala
segmento_2.find_element_by_xpath('.//div[@class="sc-eXEjpC fqUnRK"]/span[@class="sc-esjQYD dMquDU"]/time').get_attribute('datetime')

'12:55'

In [33]:
modelo_avion = segmento_2.find_element_by_xpath('//div[@class="airline-flight-details"]/span[@class="sc-gzOgki uTyOl"]').text
flota = segmento_2.find_element_by_xpath('//div[@class="airline-flight-details"]/b').text

print(f'Flota {flota} - {modelo_avion}')

Flota LA405 - Airbus 320-200


- **Tarifas**

In [34]:
driver.find_element_by_xpath('//div[@class="modal-content sc-iwsKbI eHVGAN"]//button[@class="close"]').click()

In [35]:
vuelo.click()

In [51]:
divisas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]//span[@class="currency-symbol"]')
valores = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]//span[@class="value"]')

divisas

[<selenium.webdriver.remote.webelement.WebElement (session="5feffdd59576557aa98daaa3562b2eb5", element="a12f70a1-d69c-4701-a04c-9038e62bb74a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5feffdd59576557aa98daaa3562b2eb5", element="c015e3a5-56d9-4976-bb14-073d90fc0a1e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5feffdd59576557aa98daaa3562b2eb5", element="c26c9dc4-4db6-4bcf-b2d7-677cd04a69eb")>]

In [54]:
divisa = divisas[0]
divisa.text

'US$'

In [58]:
moneda = valores[0]
print(divisa.text + moneda.text)

US$332


In [63]:
caca = []

for d in divisas:
    for v in valores:
        precio = d.text + ' ' + v.text
        caca.append(precio)
    break


tarifas = {
    'Light':caca[0],
    'Plus':caca[1],
    'Top':caca[2]
}
tarifas

{'Light': 'US$ 332', 'Plus': 'US$ 385', 'Top': 'US$ 445'}

Se encontraron 0 vuelos.
Iniciando scraping...


In [83]:
def _get_tarifa(vuelo):
	divisas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]//span[@class="currency-symbol"]')
	valor = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]//span[@class="value"]')

	precios = []

	for d in divisas:
		for v in valores:
			precio = d.text + ' ' + v.text
			precios.append(precio)
		break
	

	tarifas = {
		'Light':precios[0],
    	'Plus':precios[1],
    	'Top':precios[2]
		}

	return tarifas

def _get_escalas(vuelo):
	# Segmentos del contenido en donde se ven las escalas
	segmentos = vuelo.find_elements_by_xpath('//div[@class="sc-hZSUBg gfeULV"]/div[@class="sc-cLQEGU hyoued"]')
	escalas = len(segmentos) - 1 # Porque cuenta desde salida/escala y escala/salida
	print(f'El vuelo de MVD a MAD tiene {escalas} escala(s).')

	info_escalas = []
	for segmento in segmentos:
		# Aeropuerto de salida
		airport = segmento.find_element_by_xpath('.//div[@class="sc-iujRgT jEtESl"]/span[@class="sc-eTuwsz eumCTU"]/span[@class="sc-hXRMBi gVvErD"]').text 
		# Hora de salida
		h_salida = segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/time').get_attribute('datetime')
		# Duración de vuelo a próxima escala
		duracion = segmento.find_element_by_xpath('.//div[@class="sc-eXEjpC fqUnRK"]/span[@class="sc-esjQYD dMquDU"]/time').get_attribute('datetime')
		# Info de combinación
		try:    
		    combinacion = segmento.find_element_by_xpath('//div[@class="sc-hMFtBS cfwWiO"]/span[@class="connection-label sc-hORach NXcGo"]').text
		    cambio_avion = segmento.find_element_by_xpath('//div[@class="sc-hMFtBS cfwWiO"]/span[@class="connection-changes sc-hORach NXcGo"]').text
		    info = f'{combinacion} -- {cambio_avion}'
		except:
		    print('Revise XPath.')
		    pass
		# Flota
		flota = segmento.find_element_by_xpath('//div[@class="airline-flight-details"]/b').text
		# Modelo de avón
		modelo_avion = segmento.find_element_by_xpath('//div[@class="airline-flight-details"]/span[@class="sc-gzOgki uTyOl"]').text

		data_escalas = {
			'Aeropuerto': airport,
			'Hora de salida': h_salida,
			'Duración p/ escala': duracion,
			'Info de conexión': info,
			'Flota': flota,
			'Avión': modelo_avion
		}
		info_escalas.append(data_escalas)

	return info_escalas

def _get_tiempos(vuelo):
	
	#Hora de partida
	h_salida = vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
	#Hora de llegada
	h_llegada = vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
	# Duración de vuelo
	duracion = vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')

	data_tiempos = {
		'Hora de salida':h_salida,
		'Hora de llegada':h_llegada,
		'Duración':duracion
	}
	return data_tiempos

def main_get_info(driver):
	
	# Extraer informacion de la pagina
	vueles = vuelos
	print(f'Se encontraron {len(vuelos)} vuelos.')
	print('Iniciando scraping...')
	info = []
	for vuelo in vueles:
		tiempos = _get_tiempos(vuelo)
		
		# Click en modal para conocer escalas
		boton_escalas = vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')
		boton_escalas.click()

		# Obtener escalas
		escalas = _get_escalas(vuelo)
		
		# Cerrar modal
		cerrar_modal = driver.find_element_by_xpath('//div[@class="modal-content sc-iwsKbI eHVGAN"]//button[@class="close"]')
		cerrar_modal.click()
		vuelo.click()

		# Obtener tarifas
		tarifas = _get_tarifa(vuelo)

		data_info = {
			'Tiempos':tiempos,
			'Escalas':escalas,
			'Tarifas':tarifas
		}
		info.append(data_info)

	return info


In [84]:
main_get_info(driver)

Se encontraron 6 vuelos.
Iniciando scraping...
El vuelo de MVD a MAD tiene 1 escala(s).
El vuelo de MVD a MAD tiene 1 escala(s).
El vuelo de MVD a MAD tiene 2 escala(s).
El vuelo de MVD a MAD tiene 2 escala(s).
El vuelo de MVD a MAD tiene 2 escala(s).
El vuelo de MVD a MAD tiene 2 escala(s).


[{'Tiempos': {'Hora de salida': '11:18',
   'Hora de llegada': '12:50',
   'Duración': 'PT21H32M'},
  'Escalas': [{'Aeropuerto': 'Carrasco Intl.',
    'Hora de salida': '11:18',
    'Duración p/ escala': '2:32',
    'Info de conexión': 'Conexión en Santiago de Chile -- (cambio de avión)',
    'Flota': 'LA405',
    'Avión': 'Airbus 320-200'},
   {'Aeropuerto': 'A. Merino Benítez Intl.',
    'Hora de salida': '19:55',
    'Duración p/ escala': '12:55',
    'Info de conexión': 'Conexión en Santiago de Chile -- (cambio de avión)',
    'Flota': 'LA405',
    'Avión': 'Airbus 320-200'}],
  'Tarifas': {'Light': ' 332', 'Plus': ' 385', 'Top': ' 445'}},
 {'Tiempos': {'Hora de salida': '10:50',
   'Hora de llegada': '13:10',
   'Duración': 'PT22H20M'},
  'Escalas': [{'Aeropuerto': 'Carrasco Intl.',
    'Hora de salida': '10:50',
    'Duración p/ escala': '2:15',
    'Info de conexión': 'Conexión en San Pablo -- (cambio de avión)',
    'Flota': 'LA8047',
    'Avión': 'Airbus 320-200'},
   {'Aeropu